# Convolutional Neural Network

### Importing the libraries

In [106]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [107]:
train_datagen=ImageDataGenerator(
                          rescale=1./255,
                          zoom_range=0.2,
                          shear_range=0.2,
                          vertical_flip=True,
                          horizontal_flip=True,
                          width_shift_range=0.2,
                          height_shift_range=0.2
                      )

training_set = train_datagen.flow_from_directory(
                                                    "dataset/training_set",
                                                    target_size=(64,64),
                                                    class_mode="binary",
                                                    batch_size=32
                                                 )    

Found 8048 images belonging to 2 classes.


### Preprocessing the Test set

In [108]:
test_datagen=ImageDataGenerator(  rescale=1./255  )

test_set = test_datagen.flow_from_directory(
                                    "dataset/test_set",
                                    target_size=(64,64),
                                    class_mode="binary",
                                    batch_size=32
                                 )    

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [109]:
cnn=tf.keras.Sequential()

### Step 1 - Convolution

In [110]:
cnn.add(tf.keras.layers.Conv2D(filters=100,kernel_size=5, activation="relu",input_shape=[64,64,3]))

### Step 2 - Pooling

In [111]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Adding a second convolutional layer

In [112]:
cnn.add(tf.keras.layers.Conv2D(filters=100,kernel_size=5,activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Step 3 - Flattening

In [113]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [114]:
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))

### Step 5 - Output Layer

In [115]:
cnn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

## Part 3 - Training the CNN

### Compiling the CNN

In [116]:
cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

### Training the CNN on the Training set and evaluating it on the Test set

In [117]:
cnn.fit(x=training_set, validation_data=test_set, epochs=50)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 252 steps, validate for 63 steps
Epoch 1/50
252/252 [==============================] - 302s 1s/step - loss: 0.6964 - accuracy: 0.5017 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/50
252/252 [==============================] - 296s 1s/step - loss: 0.6933 - accuracy: 0.4945 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/50
252/252 [==============================] - 2587s 10s/step - loss: 0.6936 - accuracy: 0.5043 - val_loss: 0.6926 - val_accuracy: 0.5165
Epoch 4/50
252/252 [==============================] - 237s 942ms/step - loss: 0.6922 - accuracy: 0.5184 - val_loss: 0.6896 - val_accuracy: 0.5460
Epoch 5/50
252/252 [==============================] - 223s 887ms/step - loss: 0.6881 - accuracy: 0.5455 - val_loss: 0.6835 - val_accuracy: 0.5675
Epoch 6/50
252/252 [==============================] - 239s 950ms/step - loss: 0.6881 - accuracy: 0.5363 - val_loss: 0.6832 - val_accuracy: 0.5670
Epoch 7/50
252/252 [==================

## Part 4 - Making a single prediction

In [118]:
from keras.preprocessing import image
import numpy as np

In [120]:
image_set=image.load_img("dataset/single_prediction/cat_or_dog_1.jpg", target_size=(64,64),)
image_set=image.img_to_array(image_set)
image_set=np.expand_dims(image_set, axis=0)
result=cnn.predict(image_set)
if result[0][0]==1:
    print("dog")
else:
    print("cat")

dog
